In [58]:
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import matplotlib.colors

CMAP = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    ax.axis((xmin, xmax, ymin, ymax))


In [2]:
gdf = gp.read_file('detroit_home_values/detroit.shp')
gdf = gdf.to_crs(epsg=3857)

In [3]:
gdf.head()

,GEO_ID,NAME,YEAR,MEDIAN_HOM,geometry
0,26125132600,"Census Tract 1326, Oakland County, Michigan",2010,420200.0,POLYGON ((-9311343.965608286 5252465.629056039...
1,26163562600,"Census Tract 5626, Wayne County, Michigan",2010,240400.0,"POLYGON ((-9290914.390018923 5216535.96511464,..."
2,26163562600,"Census Tract 5626, Wayne County, Michigan",2011,232600.0,"POLYGON ((-9290914.390018923 5216535.96511464,..."
3,26163562600,"Census Tract 5626, Wayne County, Michigan",2012,230300.0,"POLYGON ((-9290914.390018923 5216535.96511464,..."
4,26163562600,"Census Tract 5626, Wayne County, Michigan",2013,215700.0,POLYGON ((-9293580.937101386 5218446.251131186...


In [4]:
gdf.pivot("GEO_ID", "YEAR", "MEDIAN_HOM").transpose().reset_index()

GEO_ID,YEAR,26087330000,26087330500,26087331500,26087332000,26087332500,26087333000,26087333500,26087334000,26087334500,...,26163596100,26163596200,26163596300,26163597000,26163985000,26163985100,26163985200,26163985400,26163985700,26163985900
0,2010,NaN,NaN,141100.0,157600.0,157000.0,196300.0,156800.0,162400.0,158700.0,...,291800.0,297800.0,363300.0,177400.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,NaN,NaN,138200.0,148900.0,136300.0,183000.0,153100.0,149300.0,153900.0,...,NaN,NaN,NaN,159700.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,NaN,NaN,134500.0,137100.0,124300.0,167100.0,139100.0,143400.0,141700.0,...,NaN,NaN,NaN,146200.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,NaN,NaN,114400.0,122800.0,113200.0,159200.0,126400.0,130100.0,136100.0,...,NaN,241800.0,NaN,123100.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,NaN,105500.0,101500.0,104500.0,117000.0,149700.0,130700.0,106900.0,127000.0,...,NaN,227700.0,294400.0,116200.0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,NaN,99700.0,101400.0,101500.0,116000.0,153600.0,121200.0,102300.0,126300.0,...,NaN,237300.0,286700.0,108100.0,NaN,NaN,NaN,NaN,NaN,NaN
6,2016,138200.0,104600.0,100900.0,107200.0,134500.0,157100.0,115100.0,114200.0,125000.0,...,NaN,230800.0,296000.0,99400.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,NaN,104600.0,108400.0,117400.0,136300.0,159100.0,123700.0,119800.0,135000.0,...,NaN,238100.0,300600.0,111700.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,158500.0,108400.0,121400.0,127900.0,146800.0,164700.0,128300.0,134500.0,143400.0,...,NaN,248100.0,314300.0,132900.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2019,158700.0,115900.0,124200.0,133600.0,144100.0,170100.0,135200.0,157700.0,156800.0,...,NaN,263500.0,328000.0,148400.0,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
def make_map(tmp_df, year):
    column = "diff"
    title = f"Median Home Value Delta (%) {year-1} to {year}"
    vmin = tmp_df[column].min()
    vmax = tmp_df[column].max()

    ax = tmp_df.loc[~tmp_df[column].isnull()].plot(
        column=column,
        cmap=CMAP, 
        scheme='quantiles',
        figsize=(25, 25),
        alpha=0.5,
        edgecolor='black',
        linewidth=1,
    )

    tmp_df.loc[tmp_df[column].isnull()].plot(
        ax=ax,
        color='black',
        figsize=(25, 25),
        alpha=0.5,
        edgecolor='black',
        linewidth=1,
    )

    add_basemap(ax, zoom=12, url=ctx.sources.ST_TONER_LITE)
    ax.set_xlim(-9.375*10**6, -9.15*10**6)
    ax.set_ylim(5.17*10**6, 5.36*10**6)
    fig = ax.get_figure()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Add color bar
#     cbax = fig.add_axes([0.95, 0.25, 0.03, 0.5])   
#     cbax.set_title("Price Delta (%)", fontsize= 15)
#     sm = plt.cm.ScalarMappable(cmap=CMAP, norm=plt.Normalize(vmin=vmin, vmax=vmax))
#     sm._A = []
#     fig.colorbar(sm, cax=cbax)

#     ax.set_title(title, fontdict={
#         'fontsize': 30, 'fontweight': 'medium'})

    plt.savefig('gif_imgs/{}_img.jpg'.format(year),  dpi=300, bbox_inches = "tight")
#     plt.show()

In [ ]:
pvt_df = gdf.pivot("GEO_ID", "YEAR", "MEDIAN_HOM").transpose().reset_index() 
cols = pvt_df.columns[1:]
for n, row in pvt_df[1:].iterrows():
    year = int(row[:1].YEAR)
    tmp_df = pd.concat([row[1:], pvt_df.iloc[n-1][1:]], axis=1).reset_index()
    print(n)
    tmp_df['diff'] = tmp_df[1+n-1]/tmp_df[0+n-1]
    tmp_df = tmp_df.merge(gdf.loc[gdf['YEAR']==year][['GEO_ID', 'geometry']], on="GEO_ID")
    tmp_df = gp.GeoDataFrame(tmp_df, crs="EPSG:3857", geometry=tmp_df.geometry)
    make_map(tmp_df, year)

In [73]:
pvt_df.to_feathereio
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

images = []
font = cv2.FONT_HERSHEY_SIMPLEX

for n, img in  enumerate(os.listdir("gif_imgs/")):
    temp = imageio.imread("gif_imgs/{}".format(img))
    temp = cv2.putText(temp, str(2011+n), (250, 250), font, 4, (0,0,0), cv2.LINE_AA)
    images.append(temp)
    print(img, n)

imageio.mimsave('Detroit.gif', images,  duration=1)

2011_img.jpg 0
2012_img.jpg 1
2013_img.jpg 2
2014_img.jpg 3
2015_img.jpg 4
2016_img.jpg 5
2017_img.jpg 6
2018_img.jpg 7
2019_img.jpg 8


In [79]:
pvt_df.to_csv('pivot_detroit.csv')